In [1]:
INFLUX_TOKEN="uwowY3xaTcBj7NBtcRH431kKkMd7bf6UN7UTFT3YX0KzOM5wbTT7hjE7Tm-aTHkVDSed-dXVzbCvxZhZ8AqA8Q=="

In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient, WriteOptions

In [3]:
path = r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\Electric_Production.csv"

In [4]:
df = pd.read_csv(path)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        397 non-null    object 
 1   IPG2211A2N  397 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.3+ KB


In [6]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [7]:
df['IPG2211A2N'] = df['IPG2211A2N'].astype(float)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATE        397 non-null    datetime64[ns]
 1   IPG2211A2N  397 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.3 KB


In [9]:
df.set_index("DATE",inplace=True)

In [10]:
df

,IPG2211A2N
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151
...,...
2017-09-01,98.6154
2017-10-01,93.6137
2017-11-01,97.3359


In [50]:
df['tag_col'] = df["IPG2211A2N"]/100

In [11]:
df

,IPG2211A2N
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151
...,...
2017-09-01,98.6154
2017-10-01,93.6137
2017-11-01,97.3359


In [13]:
INFLUX_TOKEN = "uwowY3xaTcBj7NBtcRH431kKkMd7bf6UN7UTFT3YX0KzOM5wbTT7hjE7Tm-aTHkVDSed-dXVzbCvxZhZ8AqA8Q=="
ORG = "Teknocrat_S"
BUCKET = "Time_series"
measurement = 'attempt_5'

In [14]:

with InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG, debug = True,timeout=20000) as _client:

    with _client.write_api() as _write_client:

        _write_client.write(BUCKET, ORG, record=df, data_frame_measurement_name=measurement)

>>> Request: 'POST http://localhost:8086/api/v2/write?org=Teknocrat_S&bucket=Time_series&precision=ns'
>>> Content-Type: text/plain
>>> Accept: application/json
>>> Authorization: ***
>>> User-Agent: influxdb-client-python/1.42.0
>>> Body: b'attempt_5 IPG2211A2N=72.5052 473385600000000000\nattempt_5 IPG2211A2N=70.672 476064000000000000\nattempt_5 IPG2211A2N=62.4502 478483200000000000\nattempt_5 IPG2211A2N=57.4714 481161600000000000\nattempt_5 IPG2211A2N=55.3151 483753600000000000\nattempt_5 IPG2211A2N=58.0904 486432000000000000\nattempt_5 IPG2211A2N=62.6202 489024000000000000\nattempt_5 IPG2211A2N=63.2485 491702400000000000\nattempt_5 IPG2211A2N=60.5846 494380800000000000\nattempt_5 IPG2211A2N=56.3154 496972800000000000\nattempt_5 IPG2211A2N=58.0005 499651200000000000\nattempt_5 IPG2211A2N=68.7145 502243200000000000\nattempt_5 IPG2211A2N=73.3057 504921600000000000\nattempt_5 IPG2211A2N=67.9869 507600000000000000\nattempt_5 IPG2211A2N=62.2221 510019200000000000\nattempt_5 IPG2211A2N=57.

In [15]:
client = InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG)

query_api = client.query_api()


In [59]:
flux_query = f'''
        from(bucket: "{BUCKET}")
        |> range(start: 0)
        |> filter(fn: (r) => r["_measurement"] == "{measurement}")
        '''

In [60]:
result_df = query_api.query_data_frame(flux_query)

c:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Azure_models\venv\Lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
        from(bucket: "Time_series")
        |> range(start: 0)
        |> filter(fn: (r) => r["_measurement"] == "attempt_5")
         |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunctio

In [61]:
col_value = list(pd.unique(result_df._field))[0]
col_value

'IPG2211A2N'

In [62]:
result_df

,result,table,_start,_stop,_time,_value,_field,_measurement
0,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,1985-01-01 00:00:00+00:00,72.5052,IPG2211A2N,attempt_5
1,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,1985-02-01 00:00:00+00:00,70.6720,IPG2211A2N,attempt_5
2,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,1985-03-01 00:00:00+00:00,62.4502,IPG2211A2N,attempt_5
3,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,1985-04-01 00:00:00+00:00,57.4714,IPG2211A2N,attempt_5
4,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,1985-05-01 00:00:00+00:00,55.3151,IPG2211A2N,attempt_5
...,...,...,...,...,...,...,...,...
392,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,2017-09-01 00:00:00+00:00,98.6154,IPG2211A2N,attempt_5
393,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,2017-10-01 00:00:00+00:00,93.6137,IPG2211A2N,attempt_5
394,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,2017-11-01 00:00:00+00:00,97.3359,IPG2211A2N,attempt_5
395,_result,0,1970-01-01 00:00:00+00:00,2024-05-07 06:39:49.821952+00:00,2017-12-01 00:00:00+00:00,114.7212,IPG2211A2N,attempt_5


In [63]:
result_df = result_df[["_time","_value"]]
result_df

,_time,_value
0,1985-01-01 00:00:00+00:00,72.5052
1,1985-02-01 00:00:00+00:00,70.6720
2,1985-03-01 00:00:00+00:00,62.4502
3,1985-04-01 00:00:00+00:00,57.4714
4,1985-05-01 00:00:00+00:00,55.3151
...,...,...
392,2017-09-01 00:00:00+00:00,98.6154
393,2017-10-01 00:00:00+00:00,93.6137
394,2017-11-01 00:00:00+00:00,97.3359
395,2017-12-01 00:00:00+00:00,114.7212


In [64]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   _time   397 non-null    datetime64[ns, tzutc()]
 1   _value  397 non-null    float64                
dtypes: datetime64[ns, tzutc()](1), float64(1)
memory usage: 6.3 KB


In [65]:
df_orig = pd.read_csv(path)
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        397 non-null    object 
 1   IPG2211A2N  397 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.3+ KB


In [66]:
df_orig['DATE'] = pd.to_datetime(df_orig['DATE'])
df_orig['IPG2211A2N'] = df_orig['IPG2211A2N'].astype(float)

In [67]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATE        397 non-null    datetime64[ns]
 1   IPG2211A2N  397 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.3 KB


In [68]:
result_df['_time'] = result_df['_time'].dt.tz_localize(None)

C:\Users\rajpo\AppData\Local\Temp\ipykernel_4964\1156472782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['_time'] = result_df['_time'].dt.tz_localize(None)


In [69]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   _time   397 non-null    datetime64[ns]
 1   _value  397 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.3 KB


In [70]:
df_orig

,DATE,IPG2211A2N
0,1985-01-01,72.5052
1,1985-02-01,70.6720
2,1985-03-01,62.4502
3,1985-04-01,57.4714
4,1985-05-01,55.3151
...,...,...
392,2017-09-01,98.6154
393,2017-10-01,93.6137
394,2017-11-01,97.3359
395,2017-12-01,114.7212


In [71]:
result_df.rename(columns={"_time":"DATE","_value":col_value},inplace=True)

C:\Users\rajpo\AppData\Local\Temp\ipykernel_4964\2909132196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(columns={"_time":"DATE","_value":col_value},inplace=True)


In [72]:
result_df

,DATE,IPG2211A2N
0,1985-01-01,72.5052
1,1985-02-01,70.6720
2,1985-03-01,62.4502
3,1985-04-01,57.4714
4,1985-05-01,55.3151
...,...,...
392,2017-09-01,98.6154
393,2017-10-01,93.6137
394,2017-11-01,97.3359
395,2017-12-01,114.7212


In [73]:
if df_orig.sort_index(axis=1).equals(result_df.sort_index(axis=1)):
    print("DataFrames are the same")
else:
    print("DataFrames are different")

DataFrames are the same


# Final (time range)

In [74]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient, WriteOptions

In [96]:
path = r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\AAPL.csv"
df = pd.read_csv(path)

In [6]:
INFLUX_TOKEN = "uwowY3xaTcBj7NBtcRH431kKkMd7bf6UN7UTFT3YX0KzOM5wbTT7hjE7Tm-aTHkVDSed-dXVzbCvxZhZ8AqA8Q=="
ORG = "Teknocrat_S"
BUCKET = "Time_series"
measurement = 'attempt_aapl'

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3096 non-null   int64  
 1   timestamp   3096 non-null   object 
 2   Open        3096 non-null   float64
 3   High        3096 non-null   float64
 4   Low         3096 non-null   float64
 5   Close       3096 non-null   float64
 6   Adj Close   3096 non-null   float64
 7   Volume      3096 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 193.6+ KB


In [99]:
df = df[["timestamp","Close"]]

In [100]:
flux_query = f'''
        from(bucket: "{BUCKET}")
        |> range(start: -5y)
        |> filter(fn: (r) => r["_measurement"] == "{measurement}")
        '''


In [101]:


date_formats = (
        '%b-%y','%Y-%m-%d', '%d-%m-%Y', '%m/%d/%Y', '%Y/%m/%d', '%B %d, %Y', '%d-%b-%Y','%b %d, %Y', '%Y-%m',         
        '%m/%Y','%B %Y', '%Y-%m-%d %H:%M', '%d-%m-%Y %H:%M:%S','%m/%d/%Y %I:%M %p','%Y-%m-%dT%H:%M:%S''%a, %d %b %Y %H:%M:%S GMT', '%A, %B %d, %Y'   
        )

def parse_dates(date_str):
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, exact=False)
        except ValueError:
            continue  # Continue trying other formats
    return pd.NaT

df['timestamp'] =  df['timestamp'].apply(parse_dates)

df.set_index("timestamp",inplace=True)

with InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG, debug = True,timeout=20000) as _client:

    with _client.write_api() as _write_client:

        _write_client.write(BUCKET, ORG, record=df, data_frame_measurement_name=measurement)


client = InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG)

query_api = client.query_api()

flux_query = flux_query

result_df = query_api.query_data_frame(flux_query)

col_value = list(pd.unique(result_df._field))[0]

result_df = result_df[["_time","_value"]]

result_df['_time'] = result_df['_time'].dt.tz_localize(None)

result_df.rename(columns={"_time":"DATE","_value":col_value},inplace=True)

c:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Azure_models\venv\Lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
        from(bucket: "Time_series")
        |> range(start: -5y)
        |> filter(fn: (r) => r["_measurement"] == "attempt_aapl")
         |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFu

In [102]:
result_df

,DATE,Close
0,2019-05-09,50.180000
1,2019-05-10,49.294998
2,2019-05-13,46.430000
3,2019-05-14,47.165001
4,2019-05-15,47.730000
...,...,...
1243,2024-04-17,168.000000
1244,2024-04-18,167.039993
1245,2024-04-19,165.000000
1246,2024-04-22,165.839996


In [103]:
min(result_df.DATE),max(result_df.DATE)

(Timestamp('2019-05-09 00:00:00'), Timestamp('2024-04-23 00:00:00'))

# Final (columns)

In [5]:
path = r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\AAPL.csv"
df = pd.read_csv(path)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  3096 non-null   object 
 1   Open       3096 non-null   float64
 2   High       3096 non-null   float64
 3   Low        3096 non-null   float64
 4   Close      3096 non-null   float64
 5   Adj Close  3096 non-null   float64
 6   Volume     3096 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 169.4+ KB


In [7]:
df

,timestamp,Open,High,Low,Close,Adj Close,Volume
0,03-01-2012,14.621429,14.732143,14.607143,14.686786,12.433822,302220800
1,04-01-2012,14.642857,14.810000,14.617143,14.765714,12.500644,260022000
2,05-01-2012,14.819643,14.948214,14.738214,14.929643,12.639427,271269600
3,06-01-2012,14.991786,15.098214,14.972143,15.085714,12.771555,318292800
4,09-01-2012,15.196429,15.276786,15.048214,15.061786,12.751298,394024400
...,...,...,...,...,...,...,...
3091,17-04-2024,169.610001,170.649994,168.000000,168.000000,168.000000,50901200
3092,18-04-2024,168.029999,168.639999,166.550003,167.039993,167.039993,43122900
3093,19-04-2024,166.210007,166.399994,164.080002,165.000000,165.000000,67772100
3094,22-04-2024,165.520004,167.259995,164.770004,165.839996,165.839996,48116400


In [3]:
INFLUX_TOKEN = "uwowY3xaTcBj7NBtcRH431kKkMd7bf6UN7UTFT3YX0KzOM5wbTT7hjE7Tm-aTHkVDSed-dXVzbCvxZhZ8AqA8Q=="
ORG = "Teknocrat_S"
BUCKET = "Time_series"
measurement = 'attempt_aapl_1'

In [9]:
# Define the list of columns to keep
time_col = ["_time"]
col_select = ["Open","High","Low"]
columns_to_keep = time_col + col_select

time_range = 0

# Calculate the start time for the time range
start_time = f'-{time_range}y'

# Convert the list to a formatted string
columns_string = ', '.join([f'"{col}"' for col in columns_to_keep])

# Construct the Flux query with the formatted columns string and start time
flux_query = f'''
        from(bucket: "{BUCKET}")
        |> range(start: {start_time})
        |> filter(fn: (r) => r["_measurement"] == "{measurement}")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        |> keep(columns: [{columns_string}])
        '''


In [12]:


date_formats = (
        '%b-%y','%Y-%m-%d', '%d-%m-%Y', '%m/%d/%Y', '%Y/%m/%d', '%B %d, %Y', '%d-%b-%Y','%b %d, %Y', '%Y-%m',         
        '%m/%Y','%B %Y', '%Y-%m-%d %H:%M', '%d-%m-%Y %H:%M:%S','%m/%d/%Y %I:%M %p','%Y-%m-%dT%H:%M:%S''%a, %d %b %Y %H:%M:%S GMT', '%A, %B %d, %Y'   
        )

def parse_dates(date_str):
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, exact=False)
        except ValueError:
            continue  # Continue trying other formats
    return pd.NaT

df['timsestamp'] =  df['timestamp'].apply(parse_dates)

df.set_index("timestamp",inplace=True)

with InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG, debug = True,timeout=20000) as _client:

    with _client.write_api() as _write_client:

        _write_client.write(BUCKET, ORG, record=df, data_frame_measurement_name=measurement)


client = InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG)

query_api = client.query_api()

flux_query = flux_query

result_df = query_api.query_data_frame(flux_query)



>>> Request: 'POST http://localhost:8086/api/v2/write?org=Teknocrat_S&bucket=Time_series&precision=ns'
>>> Content-Type: text/plain
>>> Accept: application/json
>>> Authorization: ***
>>> User-Agent: influxdb-client-python/1.42.0
>>> Body: b'attempt_aapl_1 Adj\\ Close=12.43382168,Close=14.6867857,High=14.7321434,Low=14.6071434,Open=14.62142944,Volume=302220800i,timsestamp="2012-01-03 00:00:00" 1330560000000000000\nattempt_aapl_1 Adj\\ Close=12.50064373,Close=14.76571369,High=14.81000042,Low=14.61714268,Open=14.6428566,Volume=260022000i,timsestamp="2012-01-04 00:00:00" 1333238400000000000\nattempt_aapl_1 Adj\\ Close=12.63942719,Close=14.92964268,High=14.94821358,Low=14.73821354,Open=14.81964302,Volume=271269600i,timsestamp="2012-01-05 00:00:00" 1335830400000000000\nattempt_aapl_1 Adj\\ Close=12.77155495,Close=15.08571434,High=15.09821415,Low=14.97214317,Open=14.991786,Volume=318292800i,timsestamp="2012-01-06 00:00:00" 1338508800000000000\nattempt_aapl_1 Adj\\ Close=12.75129795,Close=15.

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.6', 'X-Platform-Error-Code': 'invalid', 'Date': 'Wed, 08 May 2024 05:23:34 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"invalid","message":"error in building plan while starting program: cannot query an empty range"}'


In [203]:
result_df

""


In [185]:
result_df.to_csv("nb_trial.csv")

In [176]:
min(result_df._time),max(result_df._time)

(Timestamp('2019-05-09 00:00:00+0000', tz='tzutc()'),
 Timestamp('2024-04-23 00:00:00+0000', tz='tzutc()'))

In [177]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1248 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   result  1248 non-null   object                 
 1   table   1248 non-null   int64                  
 2   _time   1248 non-null   datetime64[ns, tzutc()]
 3   Close   1248 non-null   float64                
 4   Open    1248 non-null   float64                
 5   Volume  1248 non-null   int64                  
dtypes: datetime64[ns, tzutc()](1), float64(2), int64(2), object(1)
memory usage: 58.6+ KB


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient, WriteOptions

path = r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\Electric_Production.csv"
df = pd.read_csv(path)

date_formats = (
        '%b-%y','%Y-%m-%d', '%d-%m-%Y', '%m/%d/%Y', '%Y/%m/%d', '%B %d, %Y', '%d-%b-%Y','%b %d, %Y', '%Y-%m',         
        '%m/%Y','%B %Y', '%Y-%m-%d %H:%M', '%d-%m-%Y %H:%M:%S','%m/%d/%Y %I:%M %p','%Y-%m-%dT%H:%M:%S''%a, %d %b %Y %H:%M:%S GMT', '%A, %B %d, %Y'   
        )

def parse_dates(date_str):
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, exact=False)
        except ValueError:
            continue  # Continue trying other formats
    return pd.NaT

df[''] =  df[''].apply(parse_dates)

df.set_index("DATE",inplace=True)

INFLUX_TOKEN = "uwowY3xaTcBj7NBtcRH431kKkMd7bf6UN7UTFT3YX0KzOM5wbTT7hjE7Tm-aTHkVDSed-dXVzbCvxZhZ8AqA8Q=="
ORG = "Teknocrat_S"
BUCKET = "Time_series"
measurement = 'attempt_6'


with InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG, debug = True,timeout=20000) as _client:

    with _client.write_api() as _write_client:

        _write_client.write(BUCKET, ORG, record=df, data_frame_measurement_name=measurement)


client = InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG)

query_api = client.query_api()

flux_query = f'''
        from(bucket: "{BUCKET}")
        |> range(start: 0)
        |> filter(fn: (r) => r["_measurement"] == "{measurement}")
        '''

result_df = query_api.query_data_frame(flux_query)

col_value = list(pd.unique(result_df._field))[0]

result_df = result_df[["_time","_value"]]

result_df['_time'] = result_df['_time'].dt.tz_localize(None)

result_df.rename(columns={"_time":"DATE","_value":col_value},inplace=True)


# Check measurements

In [7]:
client = InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG)

In [9]:
flux_query = f'''
    import "influxdata/influxdb/schema"
    
    schema.measurements(bucket: "{BUCKET}")
'''

# Query measurements
query_api = client.query_api()
result = query_api.query(flux_query)

# Extract measurement names from the result
measurement_names = []
for table in result:
    for record in table.records:
        measurement_names.append(record.get_value())
measurement_names = [x for x in measurement_names if "teknocrat" in x.lower()]

print("Measurement names:", measurement_names)

Measurement names: ['Teknocrat_AAPL']


# Delete Measurements

In [215]:
client = InfluxDBClient(url="http://localhost:8086", token=INFLUX_TOKEN, org=ORG)

# Process Data

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient, WriteOptions

In [3]:
from new_script import Connect_InfluxDB_df

In [7]:
path = r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\workstation_data.csv"
date_col = "timestamp"
time_range = 
cols_to_keep = ["Open","High","Low"]
test  = Connect_InfluxDB_df(path,date_col,time_range)
final_result = test.run()


Data Written to Influx DB!!
Columns choosen :  ['voltage', 'current', 'power', 'frequency', 'energy', 'fp', 'ESP32_temp', 'WORKSTATION_CPU', 'WORKSTATION_CPU_POWER', 'WORKSTATION_CPU_TEMP', 'WORKSTATION_GPU', 'WORKSTATION_GPU_POWER', 'WORKSTATION_GPU_TEMP', 'WORKSTATION_RAM', 'WORKSTATION_RAM_POWER']


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.6', 'X-Platform-Error-Code': 'invalid', 'Date': 'Wed, 08 May 2024 05:33:42 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"invalid","message":"compilation failed: error @3:33-3:35: invalid expression: invalid token for primary expression: PIPE_FORWARD"}'


In [6]:
final_result

,result,table,timestamp,ESP32_temp,WORKSTATION_CPU,WORKSTATION_CPU_POWER,WORKSTATION_CPU_TEMP,WORKSTATION_GPU,WORKSTATION_GPU_POWER,WORKSTATION_GPU_TEMP,WORKSTATION_RAM,WORKSTATION_RAM_POWER,current,energy,fp,frequency,power,voltage
0,_result,0,2021-05-05,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.935144,0.184350,0.859946,59.961913,96.428881,120.036101
1,_result,0,2021-05-06,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.940018,2.522196,0.861421,59.971771,97.355351,120.045387
2,_result,0,2021-05-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.941245,4.873498,0.867491,59.969231,97.915934,120.029304
3,_result,0,2021-05-08,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.945768,7.211631,0.864681,59.975887,98.230260,120.047991
4,_result,0,2021-05-09,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.932936,9.564661,0.860642,59.952477,96.528440,120.035780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,_result,0,2021-07-29,33.810496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.668865,160.242128,0.909894,59.958156,73.332270,120.147518
84,_result,0,2021-07-30,33.901454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.677234,161.995177,0.912730,59.974823,74.476241,120.147163
85,_result,0,2021-07-31,33.840213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.687589,163.770000,0.918830,59.965603,76.133688,120.138298
86,_result,0,2021-08-01,34.241099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.694468,165.569397,0.921206,59.990071,77.135816,120.135816


# Fetch_Data

In [11]:
from fetch_data import Fetch_Data_InfluxDB

In [12]:
measurement  = "Teknocrat_AAPL"
time_range = 0
cols_to_keep = ["Open","High","Low"]
test  = Fetch_Data_InfluxDB(measurement,time_range)
final_result = test.run()

Measurement names: ['Teknocrat_AAPL']
Columns choosen :  []


c:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Azure_models\venv\Lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
            from(bucket: "Time_series")
            |> range(start: 0)
            |> filter(fn: (r) => r["_measurement"] == "Teknocrat_AAPL")
             |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message

Data Fetched from Teknocrat_AAPL


In [16]:
type(final_result)

list

In [17]:
len(final_result)

2

In [57]:
all_df = []
count = 0
for all_table in final_result:
    for table in list(pd.unique(all_table.table)):
        if count == 0:
            df1 = all_table.groupby(by="table").get_group(table)[["_time","_value","_field"]]
        else:
            df1 = all_table.groupby(by="table").get_group(table)[["_value","_field"]]
        field_name = list(pd.unique(df1._field))
        df1.rename(columns={"_value":field_name[0]},inplace=True)
        df1.drop(["_field"],axis=1,inplace=True)
        df1.reset_index(inplace=True)
        df1.drop(['index'],axis=1,inplace=True)
        all_df.append(df1)
        count = count + 1

In [58]:
all_df[1]

,Close
0,14.686786
1,14.765714
2,14.929643
3,15.085714
4,15.061786
...,...
3091,168.000000
3092,167.039993
3093,165.000000
3094,165.839996


In [59]:
type(all_df)

list

In [60]:
final_df = pd.concat(all_df,axis=1)

In [61]:
final_df

,_time,Adj Close,Close,High,Low,Open,Volume
0,2012-01-03 00:00:00+00:00,12.433822,14.686786,14.732143,14.607143,14.621429,302220800
1,2012-01-04 00:00:00+00:00,12.500644,14.765714,14.810000,14.617143,14.642857,260022000
2,2012-01-05 00:00:00+00:00,12.639427,14.929643,14.948214,14.738214,14.819643,271269600
3,2012-01-06 00:00:00+00:00,12.771555,15.085714,15.098214,14.972143,14.991786,318292800
4,2012-01-09 00:00:00+00:00,12.751298,15.061786,15.276786,15.048214,15.196429,394024400
...,...,...,...,...,...,...,...
3091,2024-04-17 00:00:00+00:00,168.000000,168.000000,170.649994,168.000000,169.610001,50901200
3092,2024-04-18 00:00:00+00:00,167.039993,167.039993,168.639999,166.550003,168.029999,43122900
3093,2024-04-19 00:00:00+00:00,165.000000,165.000000,166.399994,164.080002,166.210007,67772100
3094,2024-04-22 00:00:00+00:00,165.839996,165.839996,167.259995,164.770004,165.520004,48116400


In [62]:
df_orig = pd.read_csv(r"C:\Users\rajpo\PycharmProjects\Tecnokrat\Time-series-models\Timeseries_dashboard\AAPL.csv")
df_orig

,timestamp,Open,High,Low,Close,Adj Close,Volume
0,03-01-2012,14.621429,14.732143,14.607143,14.686786,12.433822,302220800
1,04-01-2012,14.642857,14.810000,14.617143,14.765714,12.500644,260022000
2,05-01-2012,14.819643,14.948214,14.738214,14.929643,12.639427,271269600
3,06-01-2012,14.991786,15.098214,14.972143,15.085714,12.771555,318292800
4,09-01-2012,15.196429,15.276786,15.048214,15.061786,12.751298,394024400
...,...,...,...,...,...,...,...
3091,17-04-2024,169.610001,170.649994,168.000000,168.000000,168.000000,50901200
3092,18-04-2024,168.029999,168.639999,166.550003,167.039993,167.039993,43122900
3093,19-04-2024,166.210007,166.399994,164.080002,165.000000,165.000000,67772100
3094,22-04-2024,165.520004,167.259995,164.770004,165.839996,165.839996,48116400


# Process simple

In [2]:
import pandas as pd

In [16]:
df_elec = pd.read_csv("Teknocrat_Electric_Production_ex.csv")
df_elec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   result        397 non-null    object 
 1   table         397 non-null    int64  
 2   _start        397 non-null    object 
 3   _stop         397 non-null    object 
 4   _time         397 non-null    object 
 5   _value        397 non-null    float64
 6   _field        397 non-null    object 
 7   _measurement  397 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 24.9+ KB


In [17]:
def process_data(final_result):
    all_df = []
    count = 0

    if type(final_result)==list:
        for all_table in final_result:
            for table in list(pd.unique(all_table.table)):
                if count == 0:
                    df1 = all_table.groupby(by="table").get_group(table)[["_time","_value","_field"]]
                else:
                    df1 = all_table.groupby(by="table").get_group(table)[["_value","_field"]]
                field_name = list(pd.unique(df1._field))
                df1.rename(columns={"_value":field_name[0]},inplace=True)
                df1.drop(["_field"],axis=1,inplace=True)
                df1.reset_index(inplace=True)
                df1.drop(['index'],axis=1,inplace=True)
                all_df.append(df1)
                count = count + 1
        
        final_df = pd.concat(all_df,axis=1)
        return final_df
    
    else:
        for table in list(pd.unique(final_result.table)):
            if count == 0:
                df1 = final_result.groupby(by="table").get_group(table)[["_time","_value","_field"]]
            else:
                df1 = final_result.groupby(by="table").get_group(table)[["_value","_field"]]
            field_name = list(pd.unique(df1._field))
            df1.rename(columns={"_value":field_name[0]},inplace=True)
            df1.drop(["_field"],axis=1,inplace=True)
            df1.reset_index(inplace=True)
            df1.drop(['index'],axis=1,inplace=True)
            all_df.append(df1)
            count = count + 1

        final_df = pd.concat(all_df,axis=1)
        return final_df
        

In [18]:
process_data(df_elec)

,_time,IPG2211A2N
0,01-01-1985,72.5052
1,01-02-1985,70.6720
2,01-03-1985,62.4502
3,01-04-1985,57.4714
4,01-05-1985,55.3151
...,...,...
392,01-09-2017,98.6154
393,01-10-2017,93.6137
394,01-11-2017,97.3359
395,01-12-2017,114.7212


In [8]:
df_elec

,Unnamed: 0,result,table,_start,_stop,timestamp,_value,_field,_measurement
0,0,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,1985-01-01,72.5052,IPG2211A2N,Teknocrat_Electric_Production
1,1,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,1985-02-01,70.6720,IPG2211A2N,Teknocrat_Electric_Production
2,2,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,1985-03-01,62.4502,IPG2211A2N,Teknocrat_Electric_Production
3,3,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,1985-04-01,57.4714,IPG2211A2N,Teknocrat_Electric_Production
4,4,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,1985-05-01,55.3151,IPG2211A2N,Teknocrat_Electric_Production
...,...,...,...,...,...,...,...,...,...
392,392,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,2017-09-01,98.6154,IPG2211A2N,Teknocrat_Electric_Production
393,393,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,2017-10-01,93.6137,IPG2211A2N,Teknocrat_Electric_Production
394,394,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,2017-11-01,97.3359,IPG2211A2N,Teknocrat_Electric_Production
395,395,_result,0,1970-01-01 00:00:00+00:00,2024-05-09 09:36:43.665088+00:00,2017-12-01,114.7212,IPG2211A2N,Teknocrat_Electric_Production
